# Settings

In [1]:
%env TF_KERAS = 1
import os
sep_local = os.path.sep
import sys
os.chdir('..' + sep_local+'..' + sep_local + '..' + sep_local +'..') # For Linux import
print(sep_local)
print(os.getcwd())

env: TF_KERAS=1
/
/home/azeghost/git/azeghost/Generative_Models


In [2]:
import tensorflow as tf
print(tf.__version__)

2.3.1


# Dataset loading

In [3]:
dataset_name='atari_pacman'

In [4]:
images_dir = '/home/azeghost/datasets/.mspacman/atari_v1/screens/mspacman' #Linux
validation_percentage = 25
valid_format = 'png'

In [5]:
from training.generators.from_images.file_image_generator import create_image_lists, get_generators

In [6]:
imgs_list = create_image_lists(
    image_dir=images_dir, 
    validation_pct=validation_percentage, 
    valid_imgae_formats=valid_format,
    verbose=0 
)

## downscalse the image into fifth

In [7]:
scale=1
image_size=(160//scale, 210//scale, 3)

batch_size = 10
EPIS_LEN = 10
EPIS_SHIFT = 5

inputs_shape= (EPIS_LEN, ) + image_size
latents_dim = 30
intermediate_dim = 30

In [8]:
#we created new  class_mode  episode_flat witch returns 
#da.from_array(np.reshape(batch_x, (-1,)+self.image_shape )), da.from_array(np.reshape(batch_gt, (-1,)+self.image_shape))

In [9]:
training_generator, testing_generator = get_generators(
    images_list=imgs_list, 
    image_dir=images_dir, 
    image_size=image_size, 
    batch_size=batch_size, 
    class_mode='episode',
    episode_len=EPIS_LEN, 
    episode_shift=EPIS_SHIFT
)

  INFO     | Found 1172401 training files

  INFO     | Found 381757 validation files



In [10]:
#changed output shapes of Dataset Generators 
#from (batch_size, EPIS_LEN, ) + image_size)
#to (batch_size* EPIS_LEN, ) + image_size) 

In [11]:
train_ds = tf.data.Dataset.from_generator(
    lambda: training_generator, 
    output_types=(tf.float32, tf.float32) ,
    output_shapes=(tf.TensorShape((batch_size, EPIS_LEN, ) + image_size),
                   tf.TensorShape((batch_size, EPIS_LEN, ) + image_size)
                  )
)

test_ds = tf.data.Dataset.from_generator(
    lambda: testing_generator,
    output_types=(tf.float32, tf.float32) ,
    output_shapes=(tf.TensorShape((batch_size, EPIS_LEN, ) + image_size),
                   tf.TensorShape((batch_size, EPIS_LEN, ) + image_size)
                  )
)


In [12]:
import numpy as np
from collections.abc import Iterable
if isinstance(inputs_shape, Iterable):
    _outputs_shape = np.prod(inputs_shape)


In [13]:
# intermediate_dim = 50
# inputs_shape= image_size # image shape
# latents_dim = 10

# Model's Layers definition

In [14]:
# tdConv = lambda **kwds: tf.keras.layers.TimeDistributed(tf.keras.layers.Conv2D(**kwds))
Convlstm = lambda **kwds: tf.keras.layers.ConvLSTM2D(**kwds)

In [15]:
tdDeConv = lambda **kwds: tf.keras.layers.TimeDistributed(tf.keras.layers.Conv2DTranspose(**kwds))

In [16]:
tdDense = lambda **kwds: tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(**kwds))


In [17]:
#tf.keras.layers.ConvLSTM2D

In [18]:
units=30
c=(image_size[0]//4, image_size[1]//6, intermediate_dim//2)

menc_lays = [
    Convlstm(filters=units, kernel_size=3, strides=(2, 2), activation='relu', return_sequences=True),
    Convlstm(filters=units//5, kernel_size=3, strides=(2, 3), activation='relu', return_sequences=True),
    tf.keras.layers.Flatten(),
    # No activation
    tf.keras.layers.Dense(units=latents_dim)
]

venc_lays = [
    Convlstm(filters=units, kernel_size=3, strides=(2, 2), activation='relu', return_sequences=True),
    Convlstm(filters=units//5, kernel_size=3, strides=(2, 3), activation='relu', return_sequences=True),
    tf.keras.layers.Flatten(),
    # No activation
    tf.keras.layers.Dense(units=latents_dim)
]


dec_lays = [
    tf.keras.layers.Dense(units=np.product((EPIS_LEN, ) + c), activation=tf.nn.relu),
    tf.keras.layers.Reshape(target_shape=(EPIS_LEN,) + c),
    tdDeConv(filters=units, kernel_size=3, strides=(2, 3), padding="SAME", activation='relu'),
    tdDeConv(filters=units//5, kernel_size=3, strides=(2, 2), padding="SAME", activation='relu'),
    # No activation
    tdDeConv(filters=3, kernel_size=3, strides=(1, 1), padding="SAME"),    
]


In [19]:
# units=30
# c=(image_size[0]//4, image_size[1]//6, intermediate_dim//2)
# menc_lays = [
#     tdConv(filters=units, kernel_size=3, strides=(2, 2), activation='relu'),
#     tdConv(filters=units//5, kernel_size=3, strides=(2, 3), activation='relu'), # units*9
#     tf.keras.layers.Flatten(),
#     # No activation
#     tf.keras.layers.Dense(units=latents_dim)
# ]

# venc_lays  = [
#     tdConv(filters=units, kernel_size=3, strides=(2, 2), activation='relu'),
#     tdConv(filters=units//5, kernel_size=3, strides=(2, 3), activation='relu'), # units*9
#     tf.keras.layers.Flatten(),
#     # No activation
#     tf.keras.layers.Dense(units=latents_dim)
# ]

# dec_lays = [
#     tf.keras.layers.Dense(units=np.product((EPIS_LEN,) + c), activation=tf.nn.relu),
#     tf.keras.layers.Reshape(target_shape=(EPIS_LEN,) + c),
#     tdDeConv(filters=units//5, kernel_size=3, strides=(2, 3), padding="SAME", activation='relu'),
#     tdDeConv(filters=units, kernel_size=3, strides=(2, 2), padding="SAME", activation='relu'),
    
#     # No activation
#     tdDeConv(filters=3, kernel_size=3, strides=(1, 1), padding="SAME")
# ]

# Model definition

In [20]:
dataset_name = 'atari_pacman'
model_name = dataset_name + '_VAE_TR_convlstm'

In [21]:
import os
experiments_dir=os.getcwd()+ sep_local  +'experiments'+sep_local + model_name

In [22]:
from os.path import abspath
absolute = abspath(experiments_dir)
print("Recording_dir",absolute)
print("Current working dir",os.getcwd())


Recording_dir /home/azeghost/git/azeghost/Generative_Models/experiments/atari_pacman_VAE_TR_convlstm
Current working dir /home/azeghost/git/azeghost/Generative_Models


In [23]:
#from training.autoencoders_conditional.Cond_AE import CondAE as AE
from training.autoencoding_basic.transformative.VAE import VAE as AE

In [24]:
variables_params = \
[
    {
        'name': 'inference_mean', 
        'inputs_shape':inputs_shape,
        'outputs_shape':latents_dim,
        'layers': menc_lays
    }

    ,
    
        {
        'name': 'inference_logvariance', 
        'inputs_shape':inputs_shape,
        'outputs_shape':latents_dim,
        'layers': venc_lays
    }

    ,
    
        {
        'name': 'generative', 
        'inputs_shape':latents_dim,
        'outputs_shape':inputs_shape,
        'layers':dec_lays
    }
]

In [25]:
from utils.data_and_files.file_utils import create_if_not_exist
_restore = os.path.join(experiments_dir, 'var_save_dir')
create_if_not_exist(_restore)
absolute = abspath(_restore)
print("Restore_dir",absolute)

Restore_dir /home/azeghost/git/azeghost/Generative_Models/experiments/atari_pacman_VAE_TR_convlstm/var_save_dir


In [26]:
ae = AE( 
    name=model_name,
    latents_dim=latents_dim,
    batch_size=batch_size,
    # batch_size=batch_size*EPIS_LEN,
    episode_len= 1, 
    variables_params=variables_params, 
    filepath=None #to restore trained model, set filepath=_restore
    )

  WARNING  | None
Model: "inference_mean"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_mean_inputs (Input [(None, 10, 160, 210, 3)] 0         
_________________________________________________________________
conv_lst_m2d (ConvLSTM2D)    (None, 10, 79, 104, 30)   35760     
_________________________________________________________________
conv_lst_m2d_1 (ConvLSTM2D)  (None, 10, 39, 34, 6)     7800      
_________________________________________________________________
flatten (Flatten)            (None, 79560)             0         
_________________________________________________________________
dense (Dense)                (None, 30)                2386830   
_________________________________________________________________
batch_normalization (BatchNo (None, 30)                120       
_________________________________________________________________
dropout (Dropout)            (None

In [27]:
# ae.compile(metrics=None)
ae.compile()

Model: "atari_pacman_VAE_TR_convlstm"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inference_logvariance_inputs (I [(None, 10, 160, 210 0                                            
__________________________________________________________________________________________________
inference_logvariance (Function (None, 30)           2430510     inference_logvariance_inputs[0][0
__________________________________________________________________________________________________
tf_op_layer_Mul (TensorFlowOpLa [(None, 30)]         0           inference_logvariance[0][0]      
__________________________________________________________________________________________________
tf_op_layer_Exp (TensorFlowOpLa [(None, 30)]         0           tf_op_layer_Mul[0][0]            
_______________________________________________________________________

# Callbacks

In [28]:
import logging
mpl_logger = logging.getLogger('matplotlib')
mpl_logger.setLevel(logging.WARNING)

from training.callbacks.sample_generation import SampleGeneration
from training.callbacks.save_model import ModelSaver

In [29]:
es = tf.keras.callbacks.EarlyStopping(
    monitor='loss', 
    min_delta=1e-12, 
    patience=12, 
    verbose=1, 
    restore_best_weights=False
)

In [30]:
ms = ModelSaver(filepath=_restore)

In [31]:
csv_dir = os.path.join(experiments_dir, 'csv_dir')
create_if_not_exist(csv_dir)
csv_dir = os.path.join(csv_dir, model_name+'.csv')
csv_log = tf.keras.callbacks.CSVLogger(csv_dir, append=True)
absolute = abspath(csv_dir)
print("Csv_dir",absolute)

Csv_dir /home/azeghost/git/azeghost/Generative_Models/experiments/atari_pacman_VAE_TR_convlstm/csv_dir/atari_pacman_VAE_TR_convlstm.csv


In [32]:
image_gen_dir = os.path.join(experiments_dir, 'image_gen_dir')
create_if_not_exist(image_gen_dir)
absolute = abspath(image_gen_dir)
print("Image_gen_dir",absolute)

Image_gen_dir /home/azeghost/git/azeghost/Generative_Models/experiments/atari_pacman_VAE_TR_convlstm/image_gen_dir


In [33]:
sg = SampleGeneration(latents_shape=latents_dim, filepath=image_gen_dir, gen_freq=5, save_img=True, gray_plot=True)

# Model Training

In [34]:
ae.fit(
    x=train_ds,
    input_kw=None,
    steps_per_epoch=1,
    epochs=2,
    verbose=2,
    callbacks=[ es, ms, csv_log, sg],
    workers=-1,
    use_multiprocessing=True,
    validation_data=test_ds,
    validation_steps=1
)

Epoch 1/2
{'x_logits': <tf.Tensor 'atari_pacman_VAE_TR_convlstm/tf_op_layer_x_logits/x_logits:0' shape=(10, 10, 160, 210, 3) dtype=float32>, 'z_latents': <tf.Tensor 'atari_pacman_VAE_TR_convlstm/tf_op_layer_z_latents/z_latents:0' shape=(10, 30) dtype=float32>, 'x_mean': <tf.Tensor 'atari_pacman_VAE_TR_convlstm/inference_mean/dropout/dropout/Mul_1:0' shape=(10, 30) dtype=float32>, 'x_logvariance': <tf.Tensor 'atari_pacman_VAE_TR_convlstm/inference_logvariance/dropout_1/dropout/Mul_1:0' shape=(10, 30) dtype=float32>, 'x_logpdf': <tf.Tensor 'atari_pacman_VAE_TR_convlstm/tf_op_layer_logpdf/logpdf:0' shape=(10, 30) dtype=float32>} {'x_logits': <tf.Tensor 'IteratorGetNext:2' shape=(10, 10, 160, 210, 3) dtype=float32>, 'z_latents': <tf.Tensor 'IteratorGetNext:6' shape=() dtype=float32>, 'x_logpdf': <tf.Tensor 'IteratorGetNext:3' shape=() dtype=float32>, 'x_logvariance': <tf.Tensor 'IteratorGetNext:4' shape=() dtype=float32>, 'x_mean': <tf.Tensor 'IteratorGetNext:5' shape=() dtype=float32>}

E

# Model Evaluation

## inception_score

In [ ]:
from evaluation.generativity_metrics.inception_metrics import inception_score

In [ ]:
is_mean, is_sigma = inception_score(ae, tolerance_threshold=1e-6, max_iteration=200)
print(f'inception_score mean: {is_mean}, sigma: {is_sigma}')

## Frechet_inception_distance

In [ ]:
from evaluation.generativity_metrics.inception_metrics import frechet_inception_distance

In [ ]:
fis_score = frechet_inception_distance(ae, training_generator, tolerance_threshold=1e-6, max_iteration=10, batch_size=32)
print(f'frechet inception distance: {fis_score}')

## perceptual_path_length_score

In [ ]:
from evaluation.generativity_metrics.perceptual_path_length import perceptual_path_length_score

In [ ]:
ppl_mean_score = perceptual_path_length_score(ae, training_generator, tolerance_threshold=1e-6, max_iteration=200, batch_size=32)
print(f'perceptual path length score: {ppl_mean_score}')

## precision score

In [ ]:
from evaluation.generativity_metrics.precision_recall import precision_score

In [ ]:
_precision_score = precision_score(ae, training_generator, tolerance_threshold=1e-6, max_iteration=200)
print(f'precision score: {_precision_score}')

## recall score

In [ ]:
from evaluation.generativity_metrics.precision_recall import recall_score

In [ ]:
_recall_score = recall_score(ae, training_generator, tolerance_threshold=1e-6, max_iteration=200)
print(f'recall score: {_recall_score}')

# Image Generation

## image reconstruction

### Training dataset

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from training.generators.image_generation_testing import reconstruct_from_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'reconstruct_training_images_like_a_batch_dir')
create_if_not_exist(save_dir)

reconstruct_from_a_batch(ae, training_generator, save_dir)

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'reconstruct_testing_images_like_a_batch_dir')
create_if_not_exist(save_dir)

reconstruct_from_a_batch(ae, testing_generator, save_dir)

## with Randomness

In [ ]:
from training.generators.image_generation_testing import generate_images_like_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'generate_training_images_like_a_batch_dir')
create_if_not_exist(save_dir)

generate_images_like_a_batch(ae, training_generator, save_dir)

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'generate_testing_images_like_a_batch_dir')
create_if_not_exist(save_dir)

generate_images_like_a_batch(ae, testing_generator, save_dir)

### Complete Randomness

In [ ]:
from training.generators.image_generation_testing import generate_images_randomly

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'random_synthetic_dir')
create_if_not_exist(save_dir)

generate_images_randomly(ae, testing_generator, save_dir)

### Stacked inputs outputs and predictions 

In [ ]:
from training.generators.image_generation_testing import predict_from_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'predictions')
create_if_not_exist(save_dir)

predict_from_a_batch(ae, testing_generator, save_dir)